In [6]:
%jsroot

In [7]:
    // 打开ROOT文件并获取TTree对象
    // TFile *file = TFile::Open("TriggerEvent.root");
    // TFile *file = TFile::Open("1cm_RUN2408_SB12_4h40min_TriggerEvent.root");
    // TFile *file = TFile::Open("TriggerEvent_LD_SB12_T6at9p46mK_bkg_4h40min_5kHz_2408200106.root");
    // TFile *file = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/1cmLMO_SB12_T6at9p46mK_bkg_4h40min_5kHz_2408200106/data/light.root");
    // TFile *file = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/1cmLMO_SB13_T6at8p5mK_bkg_9h20min_5kHz_2408232128/data/light.root");
    // TFile *file = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/LD_SB13_T6at8p5mK_bkg_9h20min_5kHz_2408232128/data/light.root");
    // TFile *file = TFile::Open("/mnt/d/trigger/analysis/calib/hadd/Co60_LMO.root");
    TFile *file = TFile::Open("/mnt/e/trigger/analysis/Heater/data/light.root");
    if (!file || file->IsZombie()) {
        std::cerr << "Error: Could not open file TriggerEvent.root" << std::endl;
        return 1;
    }
    TTree *tree = (TTree *)file->Get("tree1");
    if (!tree) {
        std::cerr << "Error: Could not find TTree 'tree1' in the file" << std::endl;
        return 1;
    }

    // tree->Print();

In [8]:
// 定义变量来存储Amp_filtered, RiseTime和DecayTime的值
Double_t Amp_filtered, RiseTime, DecayTime;
Long64_t MaxPos;
Double_t Baseline, Amp_raw;
Int_t nBins_Amp = 200, nBins_Risetime = 80, nBins_Decaytime = 70, nBins_Baseline = 500, nBins_time = 1E3, Amp_Max = 3E6, Amp_Min = -1E3, Bsl_Min = 2.45E6, Bsl_Max = 2.7E6;
Double_t Rise_Min = -0.3, Rise_Max = 0.4, Decay_Min = -0.05, Decay_Max = 0.1;

// 设置分支的地址
tree->SetBranchAddress("filamp", &Amp_filtered);
tree->SetBranchAddress("risetime", &RiseTime);
tree->SetBranchAddress("decaytime", &DecayTime);
tree->SetBranchAddress("baseline", &Baseline);
tree->SetBranchAddress("trigpos", &MaxPos);
tree->SetBranchAddress("rawamp", &Amp_raw);

In [9]:
// 创建画布和直方图
TCanvas *canvas = new TCanvas("canvas", "Amp_filtered", 800, 600);
TH1F *hAmpFiltered = new TH1F("hAmpFiltered", "Filtered AMP Distribution", nBins_Amp, Amp_Min, Amp_Max);

TCanvas *canvas2 = new TCanvas("canvas2", "RiseTime", 800, 600);
TH1F *hRiseTime = new TH1F("hRiseTime", "Rise Time Distribution", nBins_Risetime, Rise_Min, Rise_Max);

TCanvas *canvas3 = new TCanvas("canvas3", "DecayTime", 800, 600);
TH1F *hDecayTime = new TH1F("hDecayTime", "Decay Time Distribution", nBins_Decaytime, Decay_Min, Decay_Max);

TCanvas *canvas4 = new TCanvas("canvas4", "Baseline", 800, 600);
TH1F *hBaseline = new TH1F("hBaseline", "Baseline Distribution", nBins_Baseline, Bsl_Min, Bsl_Max);

TCanvas *canvas5 = new TCanvas("canvas5", "BLandAmpOF", 800, 600);
TH2F *hBLandAmpOF = new TH2F("hBLandAmpOF", "Baseline vs Amp_filtered", nBins_Baseline, Bsl_Min, Bsl_Max, nBins_Amp, Amp_Min, Amp_Max);

TCanvas *canvas6 = new TCanvas("canvas6", "BLandAmpRaw", 800, 600);
TH2F *hBLandAmpRaw = new TH2F("hBLandAmpRaw", "Baseline vs Amp_Raw", nBins_Baseline, Bsl_Min, Bsl_Max, nBins_Amp, Amp_Min, Amp_Max);

TCanvas *canvas7 = new TCanvas("canvas7", "BLandMaxPos", 800, 600);
TH2F *hBLandMaxPos = new TH2F("hBLandMaxPos", "Baseline vs time", nBins_time, 0, 8.56E7, nBins_Baseline, Bsl_Min, Bsl_Max);

TCanvas *canvas8 = new TCanvas("canvas8", "Amp_raw", 800, 600);
TH1F *hAmpRaw = new TH1F("hAmpRaw", "AMP Distribution", nBins_Amp, Amp_Min, Amp_Max);

TCanvas *canvas9 = new TCanvas("canvas9", "AmpandDecaytime", 800, 600);
TH2F *hAmpandDecaytime = new TH2F("hAmpandDecaytime", "Amp vs Decaytime", nBins_Amp, Amp_Min, Amp_Max, nBins_Decaytime, Decay_Min, Decay_Max);

TCanvas *canvas10 = new TCanvas("canvas10", "AmpandAmpFiltered", 800, 600);
TH2F *hAmpandAmpFiltered = new TH2F("hAmpandAmpFiltered", "Amp vs AmpFiltered", nBins_Amp, Amp_Min, Amp_Max, nBins_Amp, Amp_Min, Amp_Max);

TCanvas *canvas11 = new TCanvas("canvas11", "AmpandRisetime", 800, 600);
TH2F *hAmpandRisetime = new TH2F("hAmpandRisetime", "Amp vs Risetime", nBins_Amp, Amp_Min, Amp_Max, nBins_Risetime, Rise_Min, Rise_Max);

// 遍历TTree并填充直方图
Long64_t nentries = tree->GetEntries();
for (Long64_t i = 0; i < nentries; ++i) {
    tree->GetEntry(i);

    hAmpFiltered->Fill(Amp_filtered);
    hRiseTime->Fill(RiseTime);
    hDecayTime->Fill(DecayTime);
    hBaseline->Fill(Baseline);
    hBLandAmpOF->Fill(Baseline, Amp_filtered);
    hBLandAmpRaw->Fill(Baseline, Amp_raw);
    hBLandMaxPos->Fill(MaxPos, Baseline);
    hAmpRaw->Fill(Amp_raw);
    hAmpandDecaytime->Fill(Amp_raw, DecayTime);
    // hAmpandDecaytime->Fill(Amp_raw, RiseTime);
    hAmpandAmpFiltered->Fill(Amp_raw, Amp_filtered);
    hAmpandRisetime->Fill(Amp_raw, RiseTime);
}

// 对 hRiseTime 进行高斯拟合
TF1 *gausFitRiseTime = new TF1("gausFitRiseTime", "gaus", 0, 0.02);
// TF1 *gausFitRiseTime = new TF1("gausFitRiseTime", "[0]*exp(-0.5*((x-[1])/[2])^2) + [3]", 20, 40);
hRiseTime->Fit(gausFitRiseTime, "R");  // "R" 选项表示限制拟合范围在函数定义的范围内

// 对 hDecayTime 进行高斯拟合
TF1 *gausFitDecayTime = new TF1("gausFitDecayTime", "gaus", 0, 0.15);
// TF1 *gausFitDecayTime = new TF1("gausFitDecayTime", "[0]*exp(-0.5*((x-[1])/[2])^2) + [3]", 50, 90);
hDecayTime->Fit(gausFitDecayTime, "R");  // "R" 选项表示限制拟合范围在函数定义的范围内

// 设置直方图标题和轴标签
hAmpFiltered->SetTitle("Filtered AMP Distribution");
hAmpFiltered->GetXaxis()->SetTitle("Amp after OF");
hAmpFiltered->GetYaxis()->SetTitle("Counts");
canvas->cd();
gPad->SetLogy();
hAmpFiltered->Draw();

hRiseTime->SetTitle("Rise Time Distribution");
hRiseTime->GetXaxis()->SetTitle("RiseTime (s)");
hRiseTime->GetYaxis()->SetTitle("Counts");
canvas2->cd();
hRiseTime->Draw();
gausFitRiseTime->Draw("same");  // 绘制拟合结果

// 显示 hRiseTime 的拟合参数和统计信息
gStyle->SetOptFit(1);  // 打开拟合参数显示
TPaveStats *statsRiseTime = (TPaveStats*)hRiseTime->GetListOfFunctions()->FindObject("stats");
if (statsRiseTime) {  // 确保 statsRiseTime 不为空
    statsRiseTime->SetX1NDC(0.7); // 设置统计框的位置
    statsRiseTime->SetX2NDC(0.9);
    statsRiseTime->SetY1NDC(0.7);
    statsRiseTime->SetY2NDC(0.9);
}

hDecayTime->SetTitle("Decay Time Distribution");
hDecayTime->GetXaxis()->SetTitle("DecayTime (s)");
hDecayTime->GetYaxis()->SetTitle("Counts");
canvas3->cd();
hDecayTime->Draw();
gausFitDecayTime->Draw("same");  // 绘制拟合结果

// 显示 hDecayTime 的拟合参数和统计信息
TPaveStats *statsDecayTime = (TPaveStats*)hDecayTime->GetListOfFunctions()->FindObject("stats");
if (statsDecayTime) {  // 确保 statsDecayTime 不为空
    statsDecayTime->SetX1NDC(0.7); // 设置统计框的位置
    statsDecayTime->SetX2NDC(0.9);
    statsDecayTime->SetY1NDC(0.7);
    statsDecayTime->SetY2NDC(0.9);
}

hBaseline->SetTitle("Baseline Distribution");
hBaseline->GetXaxis()->SetTitle("Baseline value");
hBaseline->GetYaxis()->SetTitle("Counts");
canvas4->cd();
hBaseline->Draw();

hBLandAmpOF->SetTitle("Baseline vs Filtered Amp");
hBLandAmpOF->GetXaxis()->SetTitle("Baseline value");
hBLandAmpOF->GetYaxis()->SetTitle("Amp after OF");
// canvas5->cd();
// hBLandAmpOF->Draw();
hBLandAmpOF->SetMarkerStyle(20);
hBLandAmpOF->SetMarkerSize(0.8);
canvas5->cd();
gPad->SetLogz();
hBLandAmpOF->Draw("COLZ");

hBLandAmpRaw->SetTitle("Baseline vs Filtered Raw");
hBLandAmpRaw->GetXaxis()->SetTitle("Baseline value");
hBLandAmpRaw->GetYaxis()->SetTitle("Raw Amp");
// canvas6->cd();
// hBLandAmpRaw->Draw();
hBLandAmpRaw->SetMarkerStyle(20);
hBLandAmpRaw->SetMarkerSize(0.8);
canvas6->cd();
gPad->SetLogz();
hBLandAmpRaw->Draw("COLZ");

hBLandMaxPos->SetTitle("Baseline vs Time");
hBLandMaxPos->GetXaxis()->SetTitle("Time");
hBLandMaxPos->GetYaxis()->SetTitle("Baseline");
// canvas7->cd();
// hBLandMaxPos->Draw();
hBLandMaxPos->SetMarkerStyle(20);
hBLandMaxPos->SetMarkerSize(0.8);
canvas7->cd();
gPad->SetLogz();
hBLandMaxPos->Draw("COLZ");

hAmpRaw->SetTitle("AMP Distribution");
hAmpRaw->GetXaxis()->SetTitle("Raw Amp");
hAmpRaw->GetYaxis()->SetTitle("Counts");
canvas8->cd();
gPad->SetLogy();
hAmpRaw->Draw();

hAmpandDecaytime->SetTitle("Amp vs Decaytime");
// hAmpandDecaytime->SetTitle("Amp vs Risetime");
hAmpandDecaytime->GetXaxis()->SetTitle("Amp");
hAmpandDecaytime->GetYaxis()->SetTitle("Decaytime");
// hAmpandDecaytime->GetYaxis()->SetTitle("Risetime");
// canvas9->cd();
// hAmpandDecaytime->Draw();
hAmpandDecaytime->SetMarkerStyle(20);
hAmpandDecaytime->SetMarkerSize(0.8);
canvas9->cd();
gPad->SetLogz();
hAmpandDecaytime->Draw("COLZ");

hAmpandAmpFiltered->SetTitle("Amp vs Amp_filtered");
hAmpandAmpFiltered->GetXaxis()->SetTitle("Amp_raw");
hAmpandAmpFiltered->GetYaxis()->SetTitle("Amp_filtered");
hAmpandAmpFiltered->SetMarkerStyle(20);
hAmpandAmpFiltered->SetMarkerSize(0.8);
canvas10->cd();
gPad->SetLogz();
hAmpandAmpFiltered->Draw("COLZ");

hAmpandRisetime->SetTitle("Amp vs Risetime");
hAmpandRisetime->GetXaxis()->SetTitle("Amp_raw");
hAmpandRisetime->GetYaxis()->SetTitle("RiseTime");
hAmpandRisetime->SetMarkerStyle(20);
hAmpandRisetime->SetMarkerSize(0.8);
canvas11->cd();
gPad->SetLogz();
hAmpandRisetime->Draw("COLZ");

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =  1.61651e-06
NDf                       =            0
Edm                       =  1.61652e-06
NCalls                    =           66
Constant                  =      1654.54   +/-   43.4394     
Mean                      =    0.0136956   +/-   0.000118716 
Sigma                     =   0.00541703   +/-   0.000105975  	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      594.164
NDf                       =           44
Edm                       =    3.613e-06
NCalls                    =           60
Constant                  =      211.476   +/-   3.60729     
Mean                      =    0.0457497   +/-   0.000300613 
Sigma                     =    0.0200289   +/-   0.000204685  	 (limited)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas4
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas5
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas6
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas7
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas8
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas9
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas10
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas11


In [10]:
canvas->Draw();
canvas2->Draw();
canvas3->Draw();
canvas4->Draw();
canvas5->Draw();
canvas6->Draw();
canvas7->Draw();
canvas8->Draw();
canvas9->Draw();
canvas10->Draw();
canvas11->Draw();

In [11]:
Long64_t nentries = tree->GetEntries();
for (Long64_t i = 0; i < nentries; ++i) {
    tree->GetEntry(i);
        // if (Amp_raw > 6000 && Amp_filtered > 4000) 
        // {
        std::cout << MaxPos << std::endl;
    // }
}

14338
17717
24750
27628
30410
32490
35186
39421
44320
46426
47820
54937
54938
62359
66362
73200
78372
82460
89004
93160
100590
108157
114788
126045
132435
142447
146435
148220
151911
153096
156026
160540
164487
168301
174219
179167
182433
188843
193055
200832
203332
214659
214663
218598
225344
231158
235753
246036
247505
250062
256893
263016
264044
271365
273447
275553
285814
289596
292452
296330
299500
300579
303072
306381
308221
317391
320271
321657
324403
325889
328755
332180
335875
339200
350570
353086
355517
360894
365338
369131
371253
375571
378156
381736
393459
406543
410528
413749
416551
419279
421028
422337
425561
428346
432089
440143
444048
445828
448403
450530
454314
460331
464691
465893
467682
472922
481377
483917
500574
503657
513689
526248
527817
533263
535837
540170
546060
552021
553759
557548
560463
568056
578189
582443
593612
596602
602433
608003
616004
617978
621831
625133
642640
646419
650334
653711
655016
660677
664169
667386
671278
672546
678430
678436
682433
68844